<a href="https://colab.research.google.com/github/freakybobs/freakybobs/blob/master/KoboldCpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <b> [1]  If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or google will kill your api
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
# @title <b>v--Select model and then click this to start Koboldcpp (takes around 2-3 min for link to be generated)</b> {"display-mode":"form"}
import os
import time
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU because you used it to often recently, this can take a few hours before they let you back in. Try again later or subscribe to Colab Pro for immediate access. (or change email if you are really desperate)⚠️")
import time
from google.colab import runtime
from IPython.display import clear_output
import json
import re

# @markdown ### DO NOT CLOSE THIS TAB WHILE USING, GOOGLE ANTI AFK WILL KILL API

Model = "Llama3-Mullein(24B)"  # @param ["daybreak-kunoichi(7B)","Llama3-Lumimaid(8B)","Sunfall-NemoMix(12B)","Marinara-Nemomix(12B)","Llama3-Mullein(24B)"]{allow-input: true}
Layers = 99 #@param [99]{allow-input: true}
ContextSize = 4096 #@param [4096,8192] {allow-input: true}
FlashAttention = True #@param {type:"boolean"}
MakeLocalTunnelFallback = True #@param {type:"boolean"}
force_redownload_kobold = True # @param {type:"boolean"}

modellink = ''

import os
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU due to usage limits, this can take a few hours before they let you back in. Check out https://lite.koboldai.net for a free alternative (that does not provide an API link but can load KoboldAI saves and chat cards) or subscribe to Colab Pro for immediate access.⚠️")

if FlashAttention:
  FACommand = "--flashattention"
else:
  FACommand = ""

!echo Downloading KoboldCpp, please wait...
!wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux
!test -f koboldcpp_linux && echo Download Successful || echo Download Failed
!chmod +x ./koboldcpp_linux
!apt update
!apt install aria2 -y
clear_output()
if "https" in Model:
  modellink = Model
else:
  match Model:
    case "daybreak-kunoichi(7B)" :
        modellink = "https://huggingface.co/crestf411/daybreak-kunoichi-2dpo-7b-gguf/resolve/main/daybreak-kunoichi-2dpo-7b-q4_k_m.gguf"
    case "Llama3-Lumimaid(8B)" :
        modellink = "https://huggingface.co/Lewdiculous/Llama-3-Lumimaid-8B-v0.1-OAS-GGUF-IQ-Imatrix/resolve/main/v2-Llama-3-Lumimaid-8B-v0.1-OAS-Q4_K_S-imat.gguf"
    case "Sunfall-NemoMix(12B)" :
        modellink = "https://huggingface.co/Vdr1/Sunfall-NemoMix-Unleashed-12B-v0.6.1-GGUF-IQ/resolve/main/Sunfall-NemoMix-Unleashed-12B-v0.6.1-Q4_K_M.gguf"
    case "Marinara-Nemomix(12B)" :
        modellink = "https://huggingface.co/featherless-ai-quants/MarinaraSpaghetti-Nemomix-v4.0-12B-GGUF/resolve/main/MarinaraSpaghetti-Nemomix-v4.0-12B-Q4_K_S.gguf"
    case "Llama3-Mullein(24B)" :
        modellink = "https://huggingface.co/mradermacher/Llama3-24B-Mullein-v1-GGUF/resolve/main/Llama3-24B-Mullein-v1.Q4_K_S.gguf"

modelname = modellink.split('/')[-1].split('.')[0]
matched = re.search(r'/([^/]+)-GGUF/', modellink)
remodelname = ("koboldcpp/model")
if matched:
    remodelname = f"koboldcpp/{matched.group(1)}"
isModelExist = os.path.isfile('model.txt')
oldmodel = ""
if(isModelExist):
  f = open("model.txt", "r")
  oldmodel = f.read()
  print(f'oldmodel = {oldmodel}')

if(oldmodel != modellink or isModelExist == False):
  print('Download: '+ modellink)
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o model.gguf $modellink
  # !aria2c -x 16 -s 16 -k 1M --allow-overwrite="true" --summary-interval=5 $modellink -o model.gguf 2>&1 | grep -Ev 'Redirecting'
  with open('model.txt', 'w') as f:
      f.write(modellink)
clear_output()
if MakeLocalTunnelFallback:
  import urllib
  print("Trying to use LocalTunnel as a fallback tunnel (not so good)...")
  ltpw = urllib.request.urlopen('https://loca.lt/mytunnelpassword').read().decode('utf8').strip("\n")
  !nohup npx --yes localtunnel --port 5001 > lt.log 2>&1 &
  !sleep 8
  print("=================")
  print("(LocalTunnel Results)")
  !cat lt.log
  print(f"Please open the above link, and input the password '{ltpw}'\nYour KoboldCpp will start shortly...")
  print("=================")
  !sleep 10
!./koboldcpp_linux model.gguf --usecublas 0 mmq --chatcompletionsadapter AutoGuess --multiuser --gpulayers $Layers --contextsize $ContextSize --websearch --quiet --remotetunnel $FACommand